### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
!pip install langchain

In [2]:
from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [4]:
import os


In [6]:
groq_api_key=os.getenv("GROQ_API_KEY")


In [8]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 3.4 MB/s eta 0:00:00


In [10]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.29
    Uninstalling langchain-core-0.3.29:
      Successfully uninstalled langchain-core-0.3.29


In [13]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=GROQ_API_KEY)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7a77f8ba70d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7a77f89082d0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [14]:
!pip install langchain_core

In [15]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [16]:
result

AIMessage(content="French translation options:\n\n**Formal:**\n\n* Bonjour, comment allez-vous ?\n\n**Informal:**\n\n* Salut, ça va ?\n* Coucou, comment vas-tu ?  \n\nThe best option depends on the context and your relationship with the person you're speaking to. \n\n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 21, 'total_tokens': 84, 'completion_time': 0.114545455, 'prompt_time': 0.00014286, 'queue_time': 0.019783088999999997, 'total_time': 0.114688315}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bf4fb6ae-3ee1-436c-8cd8-d677be925c64-0', usage_metadata={'input_tokens': 21, 'output_tokens': 63, 'total_tokens': 84})

In [17]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

"French translation options:\n\n**Formal:**\n\n* Bonjour, comment allez-vous ?\n\n**Informal:**\n\n* Salut, ça va ?\n* Coucou, comment vas-tu ?  \n\nThe best option depends on the context and your relationship with the person you're speaking to. \n\n\n"

In [18]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'Here are a couple of ways to translate "Hello, How are you?" to French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, ça va ?** (Informal) \n\n\nLet me know if you\'d like more translations or have other phrases you want to learn!\n'

In [20]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Translate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [21]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [22]:
result.to_messages()

[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [23]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'